# Brugia Viral Search

The goal of this analysis is to identify and hopefully assemble to some degree the viruses that are found present in the Brugia RNA-Seq data.

The first step is to run kraken on the data to identify likely viruses from the RefSeq database. Given it is limited to the RefSeq database, it is surely not comprehensive enough to be extrememly beneficial in de novo assembly, so LMAT is going to be run as well.

All data for this project can be found here: /scratch/at120/brugia-rnaseq/brugia_viral_search
Currently only working with this run: 2015-07-24_H55MKBCXX

The Brugia data have been filtered out via tophat which may or may not be suffice. We will have to see

## H55MKBCXX Run

In [ ]:
qsub run-kraken.sh

Once kraken has generated output, obtaining the reads that are viral and not bactieria, arcahea, or phiX can be done with the following command

In [ ]:
grep 'Virus' kraken.translate.out | grep -v 'phiX' | cut -f1 > viral-read-names.txt

Once these read names are generate the below script can be executed to extract the reads that are either: 
A) viral and not phiX based on the kraken classification
B) not listed in the kraken output since these are "non-Brugia/wolbachia" reads.

In [ ]:
python extract-viral-reads.py

Once that's done we need to pair up the reads into pairs and orphans

I need to add the functionality to original extraction script but for now I'm going to use the khmer script

In [ ]:
module load biopython
python extract-paired-reads.py all-unmapped.r1.viral.fastq all-unmapped.r2.viral.fastq all-unmapped.r1.viral.pe.fastq all-unmapped.r2.viral.pe.fastq all-unmapped.viral.se.fastq

For the first run, I'm sending it through IDBA to see if there's anything fruitful without manipulating the data (normalizing, partioning, more filtering, etc.).

In [ ]:
qsub -v path=/scratch/at120/brugia-rnaseq/brugia_viral_search,fasta=all-unmapped.viral.interleaved.fasta run-idba.sh

## All Brugia Runs
Meanwhile, I merged all the brugia sequencing runs, ran tophat and kraken and same process as before

In [ ]:
cd /scratch/at120/brugia-rnaseq/brugia_viral_search/all-runs

cat /data/cgsb/gencore/out/Ghedin/2014-08-12_HAAULADXX/*R1* > all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2014-08-12_HAAULADXX/*R2* > all-r2.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-03-24_H2V3NBCXX/*/*n01* >> all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-03-24_H2V3NBCXX/*/*n02* >> all-r2.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-10-06_H5KNLBGXX/combined/*n01* >> all-r1.fastq.gz
cat /data/cgsb/gencore/out/Ghedin/2015-10-06_H5KNLBGXX/combined/*n02* >> all-r2.fastq.gz

qsub -v path=/scratch/at120/brugia-rnaseq/brugia_viral_search,fastq=all ../run-tophat.sh